In [1]:
%cd ..

/home/quasar/GitHub/TradeForecast


/home/quasar/GitHub/TradeForecast/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from tradeforecast import Scrapper

ticker = 'AAPL'
scrapper = Scrapper(ticker)

exported_fnames_dict = scrapper.export_historic_data()
exported_fnames_dict

[*********************100%***********************]  1 of 1 completed


{'AAPL': 'AAPL_1d_max_(None-None).csv'}

In [3]:
from tradeforecast.augmentation import DataEntryPoint, Indicators, FeatureEngg, RNNDataset, train_test_split

data_entry = DataEntryPoint(exported_fnames_dict['AAPL'])

indicators = Indicators(data_entry)
indicators.add_macd_sl().add_rsi().add_atr()

features = FeatureEngg(data_entry)
features.add_quarters().add_weeks()

lf = data_entry.data.drop_nulls()
lf.head().collect()

Datetime,Close,High,Low,Open,Volume,MACD_close_12-26,MACD-SL_close_9,RSI_close_14,ATR_close_14,Quarter_sin,Quarter_cos,Week_sin,Week_cos
"datetime[μs, EST]",f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64
1981-01-29 19:00:00 EST,0.126116,0.127232,0.126116,0.127232,46188800,-0.001314,0.000148,23.832346,0.004126,1.0,0.0,0.558647,0.829406
1981-02-01 19:00:00 EST,0.118862,0.11942,0.118862,0.11942,23766400,-0.002705,-0.000491,18.469055,0.004546,1.0,0.0,0.558647,0.829406
1981-02-02 19:00:00 EST,0.123326,0.123884,0.123326,0.123326,19152000,-0.003426,-0.001133,29.702202,0.004609,1.0,0.0,0.652822,0.757511
1981-02-03 19:00:00 EST,0.12779,0.128348,0.12779,0.12779,27865600,-0.003613,-0.001666,39.344816,0.004665,1.0,0.0,0.652822,0.757511
1981-02-04 19:00:00 EST,0.12779,0.128906,0.12779,0.12779,7929600,-0.003719,-0.0021,39.344816,0.00419,1.0,0.0,0.652822,0.757511


In [4]:
from torch.utils.data import DataLoader

dataset_kwargs = {'lf': lf,
                 'non_temporal': data_entry.non_temporal,
                 'temporal': data_entry.temporal,
                 'target': 'Close',
                 'look_back_len': 60,
                 'forecast_len': 5}

rnn_dataset = RNNDataset(**dataset_kwargs)

In [5]:
train_dataset, test_dataset = train_test_split(rnn_dataset, 0.01)
print(len(rnn_dataset), len(train_dataset), len(test_dataset))

#train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)
data_loader = DataLoader(rnn_dataset, batch_size=128, shuffle=False, drop_last=False)

10912 10803 109


In [6]:
from tradeforecast import LSTM
from torch import nn, optim

lstm_kwargs = {'input_size': len(rnn_dataset.features),
              'hidden_size': 32,
              'n_LSTM': 2,
              'fc_out_size':[],
              'output_size': rnn_dataset.forecast_len,
              'dropout': 0}

lstm_model = LSTM(**lstm_kwargs)
print(f"The device is set to ---> '{lstm_model.device}'")

The device is set to ---> 'cuda'


In [7]:
lstm_model.train_model(nn.MSELoss, optim.SGD, 1500, data_loader, 0.0005)

Epoch: [1/1500]; Loss: 7.723440
Epoch: [2/1500]; Loss: 7.614494
Epoch: [3/1500]; Loss: 7.520887
Epoch: [4/1500]; Loss: 7.430060
Epoch: [5/1500]; Loss: 7.340029
Epoch: [6/1500]; Loss: 7.249645
Epoch: [7/1500]; Loss: 7.160608
Epoch: [8/1500]; Loss: 7.072372
Epoch: [9/1500]; Loss: 6.984352
Epoch: [10/1500]; Loss: 6.895811
Epoch: [11/1500]; Loss: 6.806262
Epoch: [12/1500]; Loss: 6.715378
Epoch: [13/1500]; Loss: 6.623100
Epoch: [14/1500]; Loss: 6.529634
Epoch: [15/1500]; Loss: 6.435277
Epoch: [16/1500]; Loss: 6.340144
Epoch: [17/1500]; Loss: 6.244128
Epoch: [18/1500]; Loss: 6.147040
Epoch: [19/1500]; Loss: 6.048728
Epoch: [20/1500]; Loss: 5.949069
Epoch: [21/1500]; Loss: 5.847794
Epoch: [22/1500]; Loss: 5.744598
Epoch: [23/1500]; Loss: 5.639157
Epoch: [24/1500]; Loss: 5.531257
Epoch: [25/1500]; Loss: 5.420594
Epoch: [26/1500]; Loss: 5.306324
Epoch: [27/1500]; Loss: 5.186526
Epoch: [28/1500]; Loss: 5.059742
Epoch: [29/1500]; Loss: 4.932353
Epoch: [30/1500]; Loss: 4.803427
Epoch: [31/1500]; L

In [8]:
y, y_preds = lstm_model.test_model(test_loader)
y, y_preds

(tensor([[ 9.1013,  9.1471,  9.2320,  9.2473,  9.3198],
         [ 9.1471,  9.2320,  9.2473,  9.3198,  9.2535],
         [ 9.2320,  9.2473,  9.3198,  9.2535,  9.3685],
         [ 9.2473,  9.3198,  9.2535,  9.3685,  9.1886],
         [ 9.3198,  9.2535,  9.3685,  9.1886,  9.8581],
         [ 9.2535,  9.3685,  9.1886,  9.8581, 10.1406],
         [ 9.3685,  9.1886,  9.8581, 10.1406, 10.1964],
         [ 9.1886,  9.8581, 10.1406, 10.1964, 10.1129],
         [ 9.8581, 10.1406, 10.1964, 10.1129, 10.3123],
         [10.1406, 10.1964, 10.1129, 10.3123, 10.1988],
         [10.1964, 10.1129, 10.3123, 10.1988,  9.9788],
         [10.1129, 10.3123, 10.1988,  9.9788,  9.8743],
         [10.3123, 10.1988,  9.9788,  9.8743,  9.9053],
         [10.1988,  9.9788,  9.8743,  9.9053,  9.9497],
         [ 9.9788,  9.8743,  9.9053,  9.9497, 10.1491],
         [ 9.8743,  9.9053,  9.9497, 10.1491, 10.1897],
         [ 9.9053,  9.9497, 10.1491, 10.1897, 10.0237],
         [ 9.9497, 10.1491, 10.1897, 10.0237, 10

In [9]:
from sklearn.metrics import mean_absolute_percentage_error
from torch import Tensor

y: Tensor = y; y_preds: Tensor = y_preds
y_arr, y_preds_arr = y.cpu().numpy(), y_preds.cpu().numpy()
y_arr.shape, y_preds_arr.shape

((109, 5), (109, 5))

In [10]:
rnn_dataset.inverse_transform(y_arr), rnn_dataset.inverse_transform(y_preds_arr)

(array([[191.29001, 192.25002, 194.03   , 194.35   , 195.87   ],
        [192.25002, 194.03   , 194.35   , 195.87   , 194.48   ],
        [194.03   , 194.35   , 195.87   , 194.48   , 196.89   ],
        [194.35   , 195.87   , 194.48   , 196.89   , 193.12   ],
        [195.87   , 194.48   , 196.89   , 193.12   , 207.15001],
        [194.48   , 196.89   , 193.12   , 207.15001, 213.07   ],
        [196.89   , 193.12   , 207.15001, 213.07   , 214.24   ],
        [193.12   , 207.15001, 213.07   , 214.24   , 212.49002],
        [207.15001, 213.07   , 214.24   , 212.49002, 216.67   ],
        [213.07   , 214.24   , 212.49002, 216.67   , 214.29   ],
        [214.24   , 212.49002, 216.67   , 214.29   , 209.68001],
        [212.49002, 216.67   , 214.29   , 209.68001, 207.49   ],
        [216.67   , 214.29   , 209.68001, 207.49   , 208.14001],
        [214.29   , 209.68001, 207.49   , 208.14001, 209.07002],
        [209.68001, 207.49   , 208.14001, 209.07002, 213.25   ],
        [207.49   , 208.1

In [11]:
mean_absolute_percentage_error(y_arr, y_preds_arr)

np.float64(0.018608386747879964)

In [12]:
model_fname = lstm_model.save_model_state(ticker_interval='AAPL_1d_engg')
model_fname

'AAPL_1d_engg_LSTM_14629_5-cuda.pth'